In [30]:
import re
import pandas as pd
import numpy as np
from datetime import date
import time

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.renderers

import demoji
import emoji

In [31]:
with open('WhatsApp Chat with Sankalp 2022.txt') as f:
    lines = f.readlines()

fixed_lines = []
for i in range(len(lines)):
    if re.search("[0-9][0-9]/[0-9][0-9]/[0-9][0-9]",lines[i][:8]):
        fixed_lines.append(lines[i])
    else:
        fixed_lines[-1] = fixed_lines[-1] + " " + lines[i]
lines = fixed_lines

dict_lines = []

for line in lines:
    try:
        dict_line = {'date':line.split(",")[0],
                    'time':line.split(",")[1].split("-")[0].strip(),
                    'sender':line.split(",")[1].split("-")[1].split(":")[0].strip(),
                    'message':line.split(",")[1].split("-")[1].split(":")[1].strip()}
        dict_lines.append(dict_line)
    except:
        pass
    
df = pd.DataFrame(dict_lines)

sender_mapping = {'Anshu Malika Amex':'Anshu Malika',
                'Sobti Arjun':'Arjun Sobti',
                'Gaurav Grover EY':'Gaurav Grover',
                'Vaibhav Gulati Bau':'Vaibhav Gulati'}

df['sender'] = df['sender'].replace(sender_mapping)

df['isMedia'] = np.where(df['message']== '<Media omitted>', True, False)

df ['date'] = pd.to_datetime(df['date']).apply(lambda x: x.date())
df['day'] = df['date'].apply(lambda x: x.day)
df['time_hour'] = pd.to_datetime(df['time']).apply(lambda x: x.hour)
df['word_count'] = df['message'].apply(lambda x: len(x.split()))
df['emoji_count'] = df['message'].apply(lambda x: len(demoji.findall_list(x)))
df['emoji_used'] = df['message'].apply(lambda x: demoji.findall_list(x))
df['mentions'] = df['message'].apply(lambda x: re.findall("[@][9][1][0-9]+",x))
df['mentions_count'] = df['message'].apply(lambda x: len(re.findall("[@][9][1][0-9]+",x)))

df

,date,time,sender,message,isMedia,day,time_hour,word_count,emoji_count,emoji_used,mentions,mentions_count
0,2021-02-16,6:15 pm,Hitesh Gulati,Yahan track karenge sabke 2021 ke targets,False,16,18,7,0,[],[],0
1,2021-02-16,6:16 pm,Hitesh Gulati,Nagpal Summary as on 15th Feb\n Total km,False,16,18,8,0,[],[],0
2,2021-02-16,6:17 pm,Hitesh Gulati,@918390096090 karba hai. Ye mat sochna daaru p...,False,16,18,14,0,[],[@918390096090],1
3,2021-02-21,9:20 am,Divay Nagpal,+7 this week,False,21,9,3,0,[],[],0
4,2021-02-21,9:21 am,Divay Nagpal,<Media omitted>,True,21,9,2,0,[],[],0
...,...,...,...,...,...,...,...,...,...,...,...,...
2723,2022-09-27,11:00 pm,Aastha Gupta,So beautiful ❤️,False,27,23,3,1,[red heart],[],0
2724,2022-09-27,11:01 pm,Shubhika Sachdeva,Thank u 😊,False,27,23,3,1,[smiling face with smiling eyes],[],0
2725,2022-09-29,1:49 pm,Aastha Gupta,Kaun reh gya hai regn ke liye? Need to work lo...,False,29,13,13,0,[],[],0
2726,2022-09-29,2:42 pm,Arushi Malhotra,✅,False,29,14,1,1,[check mark button],[],0


In [58]:
# Most messages per day

message_count = df[['sender','date','message']].groupby('sender',as_index=False).agg({"date": min,"message": 'count'})
message_count['day_age'] = (date.today() - message_count['date']).apply(lambda x: x.days)
message_count['message per day'] = message_count['message'] / message_count['day_age']
message_count.sort_values(by='message per day',ascending=False)

,sender,date,message,day_age,message per day
2,Anshu Malika,2022-01-05,454,268,1.694030
5,Divay Nagpal,2021-01-03,886,635,1.395276
0,Aastha Gupta,2021-01-04,425,634,0.670347
4,Arushi Malhotra,2022-01-04,171,269,0.635688
7,Hitesh Gulati,2021-01-04,370,634,0.583596
3,Arjun Sobti,2022-01-05,154,268,0.574627
1,Agrata Kumar,2022-01-05,128,268,0.477612
9,Rahul Trehan,2022-01-02,54,271,0.199262
10,Shubhika Sachdeva,2022-01-08,47,265,0.177358
6,Gaurav Grover,2022-06-27,16,95,0.168421


In [60]:
# Best Frequency
message_count = df[['sender','date','message']].groupby('sender',as_index=False)\
    .agg(date=('date', 'min'), unique_days=('date', 'nunique'))
message_count['day_age'] = (date.today() - message_count['date']).apply(lambda x: x.days)
message_count['active frequency'] =  message_count['day_age'] / message_count['unique_days']
message_count.sort_values(by='active frequency',ascending=True)

,sender,date,unique_days,day_age,active frequency
2,Anshu Malika,2022-01-05,63,268,4.253968
5,Divay Nagpal,2021-01-03,143,635,4.440559
0,Aastha Gupta,2021-01-04,120,634,5.283333
4,Arushi Malhotra,2022-01-04,43,269,6.255814
3,Arjun Sobti,2022-01-05,40,268,6.700000
7,Hitesh Gulati,2021-01-04,93,634,6.817204
1,Agrata Kumar,2022-01-05,31,268,8.645161
6,Gaurav Grover,2022-06-27,10,95,9.500000
9,Rahul Trehan,2022-01-02,22,271,12.318182
10,Shubhika Sachdeva,2022-01-08,17,265,15.588235


In [34]:
# Media Shared
message_count = df[['sender','date','isMedia']].groupby('sender',as_index=False).agg({"date": min,"isMedia": 'sum'})
message_count['day_age'] = (date.today() - message_count['date']).apply(lambda x: x.days)
message_count['media per day'] = message_count['isMedia'] / message_count['day_age']
message_count

,sender,date,isMedia,day_age,media per day
0,Aastha Gupta,2021-01-04,15,634,0.023659
1,Agrata Kumar,2022-01-05,6,268,0.022388
2,Anshu Malika,2022-01-05,46,268,0.171642
3,Arjun Sobti,2022-01-05,13,268,0.048507
4,Arushi Malhotra,2022-01-04,12,269,0.044610
5,Divay Nagpal,2021-01-03,277,635,0.436220
6,Gaurav Grover,2022-06-27,0,95,0.000000
7,Hitesh Gulati,2021-01-04,16,634,0.025237
8,Nikhil Sharma,2021-01-04,0,634,0.000000
9,Rahul Trehan,2022-01-02,8,271,0.029520


In [35]:
# Messages per Day
day_messages = df[['day','message']].groupby('day',as_index=False).count()
day_messages

fig = px.line(day_messages, x="day", y="message", title='Messages per Day')
fig.show()

In [36]:
# Messages per Hour
day_messages = df[['time_hour','message']].groupby('time_hour',as_index=False).count()
day_messages

fig = px.line(day_messages, x="time_hour", y="message", title='Messages per Hour')
fig.show()

In [49]:
# All members word count
words = []
for message in list(df['message']):
    words = words + message.split()
words = pd.value_counts(np.array(words))
words.to_csv("words.csv")

In [56]:
sender = 'Divay Nagpal'

slice_df = df[df['sender']==sender]
words = []
for message in list(slice_df['message']):
    words = words + message.split()
pd.value_counts(np.array(words)).to_csv(sender+".csv")


In [39]:
# Talks maximum in Emojis

emoji_rate = df[['sender','word_count','emoji_count']].groupby('sender',as_index=False).sum()
emoji_rate['emoji_rate'] = emoji_rate['emoji_count'] / emoji_rate['word_count']
emoji_rate.sort_values(by='emoji_rate',ascending=False)

,sender,word_count,emoji_count,emoji_rate
11,Vaibhav Gulati,7,2,0.285714
10,Shubhika Sachdeva,183,37,0.202186
4,Arushi Malhotra,836,156,0.186603
1,Agrata Kumar,673,123,0.182764
9,Rahul Trehan,349,39,0.111748
6,Gaurav Grover,51,5,0.098039
0,Aastha Gupta,2039,158,0.077489
3,Arjun Sobti,720,49,0.068056
2,Anshu Malika,2660,173,0.065038
8,Nikhil Sharma,102,6,0.058824


In [61]:
# Average words per Day
message_count = df[['sender','date','word_count']].groupby('sender',as_index=False).agg({"date": min,"word_count": 'sum'})
message_count['day_age'] = (date.today() - message_count['date']).apply(lambda x: x.days)
message_count['words per day'] = message_count['word_count'] / message_count['day_age']
message_count.sort_values('words per day',ascending=False)

,sender,date,word_count,day_age,words per day
2,Anshu Malika,2022-01-05,2660,268,9.925373
5,Divay Nagpal,2021-01-03,5116,635,8.056693
7,Hitesh Gulati,2021-01-04,2312,634,3.646688
0,Aastha Gupta,2021-01-04,2039,634,3.216088
4,Arushi Malhotra,2022-01-04,836,269,3.107807
3,Arjun Sobti,2022-01-05,720,268,2.686567
1,Agrata Kumar,2022-01-05,673,268,2.511194
9,Rahul Trehan,2022-01-02,349,271,1.287823
10,Shubhika Sachdeva,2022-01-08,183,265,0.690566
6,Gaurav Grover,2022-06-27,51,95,0.536842


In [41]:
# Top Caller

caller = df[['sender','mentions_count']].groupby('sender',as_index=False).sum()
caller.sort_values(by='mentions_count',ascending=False)

,sender,mentions_count
0,Aastha Gupta,27
7,Hitesh Gulati,27
5,Divay Nagpal,11
4,Arushi Malhotra,10
1,Agrata Kumar,9
3,Arjun Sobti,2
2,Anshu Malika,1
9,Rahul Trehan,1
6,Gaurav Grover,0
8,Nikhil Sharma,0


In [42]:
# Favorite Emoji
emojis = []
for message in list(df['emoji_used']):
    emojis = emojis + message
emoji_df = pd.DataFrame(pd.value_counts(np.array(emojis)))
emoji_df['emoji_name'] = emoji_df.index
emoji_df.reset_index(inplace=True,drop=True)
emoji_df

,0,emoji_name
0,108,face with tears of joy
1,94,thumbs up: light skin tone
2,90,clapping hands
3,84,rolling on the floor laughing
4,65,grinning face with big eyes
...,...,...
75,1,OK hand: light skin tone
76,1,sign of the horns: medium-light skin tone
77,1,confetti ball
78,1,smiling face with horns
